In [ ]:
import base64
from urllib.parse import quote
import requests
from bs4 import BeautifulSoup as BS
from pydub import AudioSegment, silence
import os
import json

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36 Edg/127.0.0.0",
    "cookie": "_ga=GA1.1.1817152003.1724338082; pgv_pvid=4256099520; .AspNetCore.Session=CfDJ8Pc25MTayhNEuFUwBGxHcQybVNcgp6QXOvls6LLt8NiqMiM7g4bQ8IAXMYpf%2F2%2Bn0V6mQZ1Ta%2BjG1gysALpjV1FWy1lt5ddw%2F9UC9JLXqlASYCjqZ%2F%2Br6qMZ5ShiFDmzCPSQeLCu0YAHzqPbitKv6N8V1P0wItubpTyzJSimkjSe; __qc_wId=261; EudicWebSession=QYNeyJoYXNfb2xkX3Bhc3N3b3JkIjpmYWxzZSwidG9rZW4iOiJqRWZrTFZUZzVSbkJPcHdJN29XcVY5amxsemc9IiwiZXhwaXJlaW4iOjEzMTQwMDAsInVzZXJpZCI6IjM4ODU0YTcxLTU2MjYtMTFlZS1iYjUzLTAwNTA1Njg2YWU5YiIsInVzZXJuYW1lIjoi5a2Q5oGSX2RFcmMwUDlFOE1RMyIsImNyZWF0aW9uX2RhdGUiOiIyMDIzLTA5LTE4VDA1OjIxOjE0WiIsInJvbGVzIjpudWxsLCJvcGVuaWRfdHlwZSI6bnVsbCwib3BlbmlkX2Rlc2MiOm51bGwsInByb2ZpbGUiOnsibmlja25hbWUiOiLlrZDmgZIiLCJlbWFpbCI6IiIsImdlbmRlciI6IuWlsyIsInBhc3N3b3JkIjpudWxsLCJ2b2NhYnVsYXJpZXMiOnt9fSwibGFzdF9wYXNzd29yZF9jaGFuZ2VkX2RhdGUiOiI5LzE4LzIwMjMgMToyMToxNCBQTSIsInJlZGlyZWN0X3VybCI6bnVsbH0%253d; .AspNetCore.Antiforgery.W85QSzz26FE=CfDJ8Pc25MTayhNEuFUwBGxHcQwC5Eg70g99eLhWzXRc68acfd_szKjY8ZUmKWsEGLKpXEtAnrGl-uJ7_GVwRdYxJuyuXOGFSdm_0qT4t7RC-l1vhlU1O-lfgLH1MAg_qsxviF_PugzXbX3x1gqv2uE0F08; _ga_73H19QCX9F=GS1.1.1724832934.8.1.1724832935.0.0.0"
}
with open(data_file, "r") as f:
    data = json.load(f)

def fixed_encode_uri_component(n):
    return quote(n).replace('%21', '!').replace('%27', "'").replace('%28', '(').replace('%29', ')').replace('%2A', '*')

def get_txt(word):
    return fixed_encode_uri_component(base64.b64encode(word.encode()).decode())

for tense, types in data.items():
    if tense == "ip":
        pp_list = ["je", "tu", "il singulier", "nous", "vous", "ils pluriel"]
    elif tense == "imp":
        pp_list = ["tu", "nous", "vous"]
    if not os.path.exists(tense):
        os.mkdir(tense)
    for type, words in types.items():
        if not os.path.exists(os.path.join(tense, type)):
            os.mkdir(os.path.join(tense, type))
        for word in words:
            file_path = os.path.join(tense, type, word)
            if not os.path.exists(file_path):
                os.mkdir(file_path)
            if tense == "ip":
                which_cg = "1"
                segments_len = 7
            elif tense == "imp":
                which_cg = "8"
                segments_len = 4

            if type == "vp":
                origin_url = f"https://api.frdic.com/api/v2/speech/speakweb?langid=fr&txt=QYN{get_txt(word)}"
                origin_res = requests.get(origin_url, headers=headers)
                assert(origin_res.status_code == 200)
                with open("tmp.mp3", "wb") as f:
                    f.write(origin_res.content)
                audio = AudioSegment.from_mp3("tmp.mp3")
                assert(origin_res.status_code == 200)
                for index, pp in enumerate(pp_list):
                    pp_path = os.path.join("pp", pp + ".mp3")
                    pp_audio = AudioSegment.from_mp3(pp_path)
                    file = os.path.join(file_path, "q" + str(index + 1) + "_" + word + " (" + pp + ")" + ".mp3")
                    (audio + pp_audio).export(file, format='mp3')

                word = word.split(" ")[1]
                base_url = f"https://www.frdic.com/dicts/cg/{word}?forcecg=true"
                base_res = requests.get(base_url, headers=headers)
                assert(base_res.status_code == 200)
                base_soup = BS(base_res.content, "html.parser")

                page_status = base_soup.find("input", {"type": "hidden", "id": "page-status"})
                assert(page_status.has_attr("value"))
                page_url = f"https://www.frdic.com/Dicts/TabWithCgIdx"
                data = {
                    "whichcg": which_cg,
                    "pagestatus": page_status["value"],
                }
                page_res = requests.get(page_url, headers=headers, data=data)
                assert(page_res.status_code == 200)
                page_soup = BS(page_res.content, "html.parser")

                word_div = page_soup.find("div", {"id": "elCgMainChild", "class": "child"})
                words_array = [""]
                index = 0
                for element in word_div:
                    if element.name == "br":
                        words_array.append("")
                        index += 1
                    else:
                        words_array[index] += element.text
                words_array.insert(0, word)
                words_array = [word.strip() for word in words_array]
                for index, phrase in enumerate(words_array):
                    if index == 0:
                        continue
                    phrase_array = phrase.split(" ")
                    if phrase_array[0] == "Je":
                        pr = "me"
                    elif phrase_array[0] == "Tu":
                        pr = "te"
                    elif phrase_array[0] == "Il" or phrase_array[0] == "Ils":
                        pr = "se"
                    elif phrase_array[0] == "Nous":
                        pr = "nous"
                    elif phrase_array[0] == "Vous":
                        pr = "vous"
                    phrase_array.insert(1, pr)
                    phrase = " ".join(phrase_array)
                    audio_url = f"https://api.frdic.com/api/v2/speech/speakweb?langid=fr&txt=QYN{get_txt(phrase)}"
                    print(audio_url)
                    audio_res = requests.get(audio_url, headers=headers)
                    assert(audio_res.status_code == 200)
                    file = os.path.join(file_path, "a" + str(index) + "_" + phrase + ".mp3")
                    with open(file, "wb") as f:
                        f.write(audio_res.content)
            else:
                base_url = f"https://www.frdic.com/dicts/cg/{word}?forcecg=true"
                base_res = requests.get(base_url, headers=headers)
                assert(base_res.status_code == 200)
                base_soup = BS(base_res.content, "html.parser")

                page_status = base_soup.find("input", {"type": "hidden", "id": "page-status"})
                assert(page_status.has_attr("value"))
                page_url = f"https://www.frdic.com/Dicts/TabWithCgIdx"
                data = {
                    "whichcg": which_cg,
                    "pagestatus": page_status["value"],
                }
                page_res = requests.get(page_url, headers=headers, data=data)
                assert(page_res.status_code == 200)
                page_soup = BS(page_res.content, "html.parser")

                word_div = page_soup.find("div", {"id": "elCgMainChild", "class": "child"})
                words_array = [""]
                index = 0
                for element in word_div:
                    if element.name == "br":
                        words_array.append("")
                        index += 1
                    else:
                        words_array[index] += element.text
                words_array.insert(0, word)
                words_array = [word.strip() for word in words_array]

                tense_button = page_soup.find("a", {"class": "voice-js voice-button", "title": "变位发音"})
                assert(tense_button.has_attr("data-rel"))
                tense_url = f"https://api.frdic.com/api/v2/speech/speakweb?{tense_button['data-rel']}"
                tense_res = requests.get(tense_url, headers=headers)
                assert(tense_res.status_code == 200)

                with open("tmp.mp3", "wb") as f:
                    f.write(tense_res.content)

                audio = AudioSegment.from_mp3("tmp.mp3")
                segments = silence.split_on_silence(audio, silence_thresh=-40, min_silence_len=500)
                assert(len(segments) == segments_len)

                for i, segment in enumerate(segments):
                    if i == 0:
                        for j, pp in enumerate(pp_list):
                            pp_path = os.path.join("pp", pp + ".mp3")
                            pp_audio = AudioSegment.from_mp3(pp_path)
                            segment_file = os.path.join(file_path, "q" + str(j + 1) + "_" + words_array[i] + " (" + pp + ")" + ".mp3")
                            (segment + pp_audio).export(segment_file, format='mp3')
                    else:
                        segment_file = os.path.join(file_path, "a" + str(i) + "_" + words_array[i] + ".mp3")
                        segment.export(segment_file, format='mp3')
os.remove("tmp.mp3")

In [ ]:
import base64
import requests
import os
from urllib.parse import quote

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36 Edg/127.0.0.0",
}

def fixed_encode_uri_component(n):
    return quote(n).replace('%21', '!').replace('%27', "'").replace('%28', '(').replace('%29', ')').replace('%2A', '*')

def get_txt(word):
    return fixed_encode_uri_component(base64.b64encode(word.encode()).decode())

if not os.path.exists("pp"):
    os.mkdir("pp")

word_list = ["je", "tu", "il singulier", "nous", "vous", "ils pluriel"]

for word in word_list:
    file_path = os.path.join("pp", word + ".mp3")

    url = f"https://api.frdic.com/api/v2/speech/speakweb?langid=fr&txt=QYN{get_txt(word)}"
    print(url)
    res = requests.get(url, headers=headers)
    assert(res.status_code == 200)

    with open(file_path, "wb") as f:
        f.write(res.content)